# These notebooks are here to help you visualize and manipulate the python scripts

## qualtrics_to_matlab

In [10]:
# ------------------------------------------------------------------------------
# import packages --------------------------------------------------------------
# ------------------------------------------------------------------------------

import os
import requests
import json
import sys
import pandas as pd
import numpy as np


# ------------------------------------------------------------------------------
# settings ---------------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- find your Qualtrics IDs
# https://www.qualtrics.com/support/integrations/api-integration/finding-qualtrics-ids/

qualtrics_data_center = 'YOUR DATA CENTER' # example: duke.qualtrics.com
api_token = 'YOUR API TOKEN' # example: NEeEIn2Pr7CKsMOKPq5ecgXXxpWJkljPuJeJAYnF
survey_id = 'YOUR SURVEY ID' # example: SV_eBvCXzEVA9KoIPr


# ------------------------------------------------------------------------------
# inputs -----------------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- in the MATLAB script, we provide the qualtrics reponse ID as an input variable;
# ---- here, we need to define it ourselves

# qualtrics_response_id = sys.argv[1]
qualtrics_response_id = 'ENTER QUALTRICS RESPONSE ID HERE' # example: R_baxKuAaVeprJ56R

# ------------------------------------------------------------------------------
# call Qualtrics API -----------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- define url for request call
url = 'https://' + qualtrics_data_center + '/API/v3/surveys/' +\
        survey_id + '/responses/' + str(qualtrics_response_id)
# ---- get data from API
response = requests.get(url, headers={'X-API-TOKEN': api_token}).text


# ------------------------------------------------------------------------------
# reformat data ----------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- convert text string dictionary
response = json.loads(response)
# ---- get past the metadata/move down into the survey results (flattens dictionary)
response = response['result']['values']
# ---- convert flat dictionary into pandas Series
response = pd.Series(response)
# ---- get the question responses + ID variables
subject_id = response['subject_id']
experimental_condition = response['experimental_condition']
ratings = response[['QID1_' + str(i) for i in range(1,6)]]
# ---- replace qualtrics question IDs with meaningful ones
ratings.index = ['Apples', 'Gummies', 'KitKats', 'PopChips', 'Beets']
# ---- reformat so it is easy for readtable to work in MatLab
output = pd.DataFrame({'subject_id': subject_id,
                       'experimental_condition': experimental_condition,
                       'foods': ratings.index,
                       'ratings': ratings}).set_index('subject_id')


# ------------------------------------------------------------------------------
# export data ------------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- get the directory one level up from the current file
directory = os.path.dirname(os.path.abspath(''))
# ---- get the filepath for the current subject's raw data
export_path = os.path.join(directory, 'raw_data', 'sub'+str(response.subject_id))
# ---- create a csv so matlab can read it
output.to_csv(os.path.join(export_path, 'food_ratings.csv'))


## matlab_to_qualtrics

In [ ]:
# ------------------------------------------------------------------------------
# import packages --------------------------------------------------------------
# ------------------------------------------------------------------------------

import base64
import webbrowser
import json
import sys
import pandas as pd
import numpy as np


# ------------------------------------------------------------------------------
# settings ---------------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- find your Qualtrics IDs
# https://www.qualtrics.com/support/integrations/api-integration/finding-qualtrics-ids/

qualtrics_data_center = 'YOUR DATA CENTER' # example: duke.qualtrics.com
survey_id = 'YOUR SURVEY ID' # example: SV_eBvCXzEVA9KoIPr


# ------------------------------------------------------------------------------
# inputs -----------------------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- in the MATLAB script, we provide the qualtrics reponse ID as an input variable;
# ---- here, we need to define it ourselves

# sub_id = sys.argv[1]
# experimental_condition = sys.argv[2]
# food1 = sys.argv[3]
# food2 = sys.argv[4]

sub_id = '0001'
experimental_condition = 'control'
food1 = 'Apples'
food2 = 'Beets'

# ------------------------------------------------------------------------------
# format data for URL (needs to be flat) ---------------------------------------
# ------------------------------------------------------------------------------

# ---- IMPORTANT: in order for variables to get passed to Qualtrics, you must
# ---- define them as embedded variables. In my example, subject_id, experimental_condition,
# ---- food1, and food2 are all defined in the survey.
# ---- https://www.qualtrics.com/support/survey-platform/survey-module/survey-flow/standard-elements/embedded-data/

# create a flat series
json_payload = pd.Series({'subject_id': sub_id,
                          'experimental_condition': experimental_condition,
                          'food1': food1,
                          'food2': food2})


# ------------------------------------------------------------------------------
# create link from data --------------------------------------------------------
# ------------------------------------------------------------------------------

# ---- encode the json file to be added to the URL
encoded_payload = base64.b64encode(bytes(json_payload.to_json(), 'utf-8')).decode('utf-8')
# ---- concatenate strings to generate a final link (as a str)
link = 'https://' + qualtrics_data_center + '/jfe/form/' +\
        survey_id + '?Q_EED=' + encoded_payload
# ---- open link in browser
webbrowser.open_new(link)